In [1]:
import pandas as pd
import east

In [8]:
from east.asts import base
import re

In [21]:
data = pd.read_csv('./Downloads/cw_5.csv', sep=';', encoding='cp1251')

In [22]:
data.text = data.text.str.replace('"Ранее вы обращались по вопросу: "', '').str.replace(',Ответ специалиста:','')\
            .str.replace('Мы пришлём вам уведомление и обязательно решим ваш вопрос.', '')\
            .str.replace('Я передал вопрос сотруднику службы поддержки.', '')\
            .str.replace('Извините, сейчас обращений много, поэтому он вернется с ответом в течение', '')        

In [23]:
data = data.groupby(['ticketid', 'Th_new'])['text'].apply(lambda x: '. '.join(x)).reset_index()

In [28]:
data.Th_new = data.Th_new.str.replace('Озон аннулировал товар', 'Аннуляция заказа')

In [29]:
data.groupby(['Th_new'])['ticketid'].count()

Th_new
Акция не заведена / не смогли найти в системе                         284
Аннуляция заказа                                                     3409
Деньги начислены на ПС                                                 75
Жалоба на минимальную стоимость заказа                                 48
Не могу оплатить заказ                                                282
Не смог вернуть товар / нарушен срок забора                            88
Не сработала акция (скидка / промокод / баллы) / Премиум подписка     328
Нет оплаты при получении / Ограничение по постоплате                  162
Ошибочно / не начислены деньги (баллы)                                129
Просьба отписать от рекламных рассылок                                195
Name: ticketid, dtype: int64

In [31]:
from east.asts import base
from east import utils

In [32]:
cl = re.compile(r'[^A-Za-z0-9 \-\n\r.,;!?А-Яа-я]+')
data.text = [re.sub(cl,' ',x.replace('\n',' ')).lower() for x in data.text.values[::]]

In [58]:
data_test = data [-1000:]
data_x = data[:-1000]
ast=[]

In [41]:
for el in data.Th_new.unique():
    ast.append(base.AST.get_ast(data_x.loc[data_x.Th_new == el].text.tolist()))

In [51]:
data_x.loc[0].ticketid

55114209

In [75]:
res = []
for i in range(len(data_test)):
    a = [data_test.iloc[i].ticketid, data_test.iloc[i].Th_new, data_test.iloc[i].text]
    arr = []
    for asti in ast:
        arr.append(asti.score(data_test.iloc[i].text))
    a += arr
    res.append([a])

In [96]:
res_m = []

In [97]:
for i in range(1,101):
    res_m.append(res[(i-1)*13:i*13])

In [100]:
res_matrix = pd.DataFrame(res_m)

In [102]:
res_matrix.columns = ['ticketid', 'Theme', 'text', 'Rel1', 'Rel2', 'Rel3', 'Rel4', 'Rel5', 'Rel6', 'Rel7', 'Rel8', 'Rel9', 'Rel10']

In [103]:
res_matrix.head(10)

,ticketid,Theme,text,Rel1,Rel2,Rel3,Rel4,Rel5,Rel6,Rel7,Rel8,Rel9,Rel10
0,55215790,Аннуляция заказа,были отменены следующие позиции - картина по н...,0.233742,0.224178,0.227091,0.227965,0.305114,0.213227,0.264806,0.246957,0.278878,0.257489
1,55215804,Аннуляция заказа,нет спасибо!!! только ещ бонусами всего добр...,0.250961,0.248162,0.261630,0.237878,0.259909,0.229131,0.230361,0.234742,0.247157,0.218321
2,55215834,Аннуляция заказа,часа. мы пришл м вам уведомление.. здравству...,0.207183,0.195226,0.216350,0.206465,0.253886,0.185727,0.212587,0.180453,0.215297,0.209676
3,55215860,Ошибочно / не начислены деньги (баллы),"если вы оформили возврат товаров из заказа, и ...",0.217058,0.205104,0.209807,0.223039,0.307674,0.195035,0.270013,0.260276,0.300128,0.272498
4,55215934,Аннуляция заказа,"спасибо, но масло я отменил сам и уже давно. в...",0.196108,0.178920,0.188384,0.185249,0.224247,0.186453,0.185086,0.192379,0.196357,0.168425
5,55215961,Не могу оплатить заказ,"он приш л один раз. первоначально я его ввела,...",0.217973,0.196012,0.232956,0.205234,0.239853,0.200635,0.194428,0.189773,0.203781,0.184262
6,55215962,Не сработала акция (скидка / промокод / баллы)...,"понятно. александра, к сожалению, нет.. сейчас...",0.224484,0.202606,0.209169,0.226713,0.226001,0.189271,0.194806,0.191549,0.199398,0.198307
7,55215972,Аннуляция заказа,"наверное. здравствуйте, юлия. ожидайте 6-10 ми...",0.245456,0.244956,0.238912,0.244109,0.276659,0.227204,0.239737,0.235038,0.248786,0.234924
8,55216022,Акция не заведена / не смогли найти в системе,вам спасибо за обращение. всего хорошего.. спа...,0.244784,0.211412,0.257241,0.245794,0.247389,0.217459,0.228594,0.232618,0.241218,0.209904
9,55216050,Не сработала акция (скидка / промокод / баллы)...,"здравствуйте, есть кто-нибудь?. ожидайте неско...",0.219788,0.201567,0.212542,0.222314,0.232385,0.180889,0.182551,0.190590,0.204641,0.199829
